<a href="https://colab.research.google.com/github/LeeJungtae/TensorflowProject1/blob/main/TensorflowProject4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

저번에 했던 실습과 컨셉은 동일하다.\
하지만 이번에는 케라스와 텐서플로우 허브를 사용해 분류해보도록 하자.

In [3]:
import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("버전:", tf.__version__)
print("즉시 실행모드:", tf.executing_eagerly())
print("허브 버전:", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

버전: 2.4.1
즉시 실행모드: True
허브 버전: 0.12.0
GPU 사용 불가능


IMDB 데이터셋은 imdb reviews 또는 텐서플로우 데이터셋에 포함되어 있다.

In [4]:
# 훈련 세트를 6대 4로 나눈다.
# 결국 훈련에 15,000개 샘플, 검증에 10,000개 샘플, 테스트에 25,000개 샘플을 사용하게 된다.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True
)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFCU92A/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFCU92A/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFCU92A/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


첫 10개의 샘플 데이터를 출력해보자.

In [5]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

처음 10개의 레이블로 출력해보자.

In [6]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

계속 반복 학습해보자.\
신경망은 층을 쌓아서 만든다.\
여기에는 세 개의 중요한 구조적 결정이 필요하다.

- 어떻게 텍스트를 표현할 것인가?

- 모델에서 얼마나 많은 층을 사용할 것인가?

- 각층에서 얼마나 많은 은닉 유닛(hidden unit)을 사용할 것인가?

텍스트를 표현하는 한 가지 방법은 문장을 임베딩(embedding) 벡터로 바꾸는 것이다.\
첫 번째 층으로 사전 훈련(pre-trained)된 텍스트 임베딩을 사용할 수 있다.\
두 가지 장점이 있다.

- 텍스트 전처리에 대해 신경 쓸 필요가 없다.

- 전이 학습의 장점을 이용한다.

- 임베딩은 고정 크기이기 때문에 처리 과정이 단순해진다.

이번 예제에서는 텐서플로우 허브에 있는 사전 훈련된 텍스트 임베딩 모델인 google/tf2-preview/gnews-swivel-20dim/1을 사용하려고 한다.

먼저 문장을 임베딩시키기 위해 텐서플로우 허브 모델을 사용하는 케라스 층을 만들어보자.

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

이제 전체 모델을 만들어보자.

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


순서대로 층을 쌓아 분류기를 만든다.

1. 첫 번째 층은 텐서플로우 허브 층이다.

2. 이 고정 크기의 출력 벡터는 16개의 은닉 유닛(hidden unit)을 가진 완전 연결 층(Dense)으로 주입된다.

3. 마지막 층은 하나의 출력 노드를 가진 완전 연결 층이다. sigmoid 활성화 함수를 사용하므로 확률 또는 신뢰도 수준을 표현하는 0~1 사이의 실수가 출력된다.

이제 모델을 컴파일해보자.

다시 한 번 복습 타임.\
모델이 훈련하려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요하다.\
이 예제는 이진 분류 문제이고 모델이 확률을 출력하므로 binary_crossentropy 손실 함수를 사용하겠다.

이제 모델이 사용할 옵티마이저와 손실함수를 설정해보자.

In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

이 모델을 512개의 샘플로 이뤄진 미니배치(mini-batch)에서 20번의 에포크(epoch) 동안 훈련한다.\
x_train과 y_train 텐서에 있는 모든 샘플에 대해 20번 반복한다.\
훈련하는 동안 10,000개의 검증 세트에서 모델의 손실과 정확도를 모니터링한다.

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 105ms/step - loss: 0.8899 - accuracy: 0.4921 - val_loss: 0.6846 - val_accuracy: 0.5388
Epoch 2/20
30/30 [==============================] - 4s 100ms/step - loss: 0.6594 - accuracy: 0.5841 - val_loss: 0.6387 - val_accuracy: 0.6015
Epoch 3/20
30/30 [==============================] - 3s 96ms/step - loss: 0.6256 - accuracy: 0.6191 - val_loss: 0.6120 - val_accuracy: 0.6262
Epoch 4/20
30/30 [==============================] - 3s 96ms/step - loss: 0.5983 - accuracy: 0.6469 - val_loss: 0.5871 - val_accuracy: 0.6682
Epoch 5/20
30/30 [==============================] - 3s 96ms/step - loss: 0.5679 - accuracy: 0.6823 - val_loss: 0.5622 - val_accuracy: 0.6952
Epoch 6/20
30/30 [==============================] - 3s 94ms/step - loss: 0.5406 - accuracy: 0.7098 - val_loss: 0.5370 - val_accuracy: 0.7219
Epoch 7/20
30/30 [==============================] - 3s 99ms/step - loss: 0.5111 - accuracy: 0.7346 - val_loss: 0.5090 - val_accuracy: 0.7386
Epoch 8/20


모델의 성능을 확인해보자.\
두 개의 값이 반환된다.\
손실과 정확도이다.

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3083 - accuracy: 0.8637
loss: 0.308
accuracy: 0.864


이 예제는 매우 단순한 방식을 사용하므로 86% 정도의 정확도를 달성했다.\
고급 방법을 사용한 모델은 95%에 가까운 정확도를 얻는다.